/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Geja3K4l4Sdyl0Yb5sBRHEQWzP9d-Yh2
From (redirected): https://drive.google.com/uc?id=1Geja3K4l4Sdyl0Yb5sBRHEQWzP9d-Yh2&confirm=t&uuid=649d9ee3-9be1-4ae0-867a-777e81ab55d2
To: /content/dataset.zip
100% 1.15G/1.15G [00:17<00:00, 66.5MB/s]


# Import library yang dibutuhkan

In [8]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import osimport tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np



# Preprocessing Data

## Load & Organisasi Dataset



dataset link drive : https://drive.google.com/file/d/1Geja3K4l4Sdyl0Yb5sBRHEQWzP9d-Yh2/view

In [ ]:
!pip install -q gdown

file_id = "1Geja3K4l4Sdyl0Yb5sBRHEQWzP9d-Yh2"
!gdown --id {file_id} -O dataset.zip

import zipfile

with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")  # semua file ke folder 'dataset'

base_dir = "/content/dataset/dataset_skin_capstone"


 Image Augmentation dan Normalisasi

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)



In [ ]:
images, labels = next(train_generator)

num_images = 9
class_names = list(train_generator.class_indices.keys())

plt.figure(figsize=(10, 10))
for i in range(num_images):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[np.argmax(labels[i])])
    plt.axis('off')
plt.suptitle("Contoh Gambar dari Train Generator (Dengan Augmentasi)", fontsize=16)
plt.tight_layout()
plt.show()


# Pembangunan Arsitektur Model cnn dan latih dari Awal

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 4 kelas
])


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3501 - loss: 1.7879 - val_accuracy: 0.3712 - val_loss: 1.0989
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4450 - loss: 1.0259 - val_accuracy: 0.3579 - val_loss: 1.0816
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.4944 - loss: 1.0190 - val_accuracy: 0.4281 - val_loss: 1.0876
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.5189 - loss: 0.9994 - val_accuracy: 0.4181 - val_loss: 1.0816
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.4997 - loss: 0.9799 - val_accuracy: 0.5017 - val_loss: 0.9996
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.5383 - loss: 0.9593 - val_accuracy: 0.3946 - val_loss: 1.1028
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.5280 - loss: 0.9372 - val_accuracy: 0.3913 - val_loss: 1.1278
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.4915 - loss: 0.9716 - val_accuracy: 0.4281 - val_loss:

# Pelatihan Model (Training)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

 # Evaluasi Model

In [ ]:
loss, acc = model.evaluate(val_generator)
print(f"Akurasi validasi: {acc * 100:.2f}%")

# Penyempurnaan Model Transfer learning

In [7]:
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)



16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.3401 - loss: 1.1448 - val_accuracy: 0.3077 - val_loss: 1.1273
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3379 - loss: 1.1143 - val_accuracy: 0.3311 - val_loss: 1.1070
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3021 - loss: 1.1084 - val_accuracy: 0.3311 - val_loss: 1.1049
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.3122 - loss: 1.1055 - val_accuracy: 0.3344 - val_loss: 1.1015
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3265 - loss: 1.1035 - val_accuracy: 0.3344 - val_loss: 1.0987


# Penyempurnaan Model Fine Tuning

In [9]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.3451 - loss: 1.1884 - val_accuracy: 0.3344 - val_loss: 1.0991
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3461 - loss: 1.1015 - val_accuracy: 0.3344 - val_loss: 1.0995
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.3018 - loss: 1.1202 - val_accuracy: 0.3344 - val_loss: 1.1002
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3316 - loss: 1.1116 - val_accuracy: 0.3344 - val_loss: 1.1008
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.3334 - loss: 1.1078 - val_accuracy: 0.3344 - val_loss: 1.1010
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.3593 - loss: 1.1114 - val_accuracy: 0.3344 - val_loss: 1.1008
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.3195 - loss: 1.1216 - val_accuracy: 0.3344 - val_loss: 1.1005
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.3019 - loss: 1.1079 - val_accuracy: 0.3344 - val_loss:

In [10]:


pred_probs = model.predict(test_generator)

y_pred = np.argmax(pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())



cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

plt.figure(figsize=(10, 8))
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()



NameError: name 'test_generator' is not defined

# Penyempurnaan Model Fine Tuning menggunakan Vit

In [11]:
pip install tensorflow tensorflow_hub


In [12]:
import tensorflow_hub as hub

In [16]:
VIT_URL = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"
IMG_SIZE = 224
vit_feature_extractor = hub.KerasLayer(VIT_URL, trainable=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


In [17]:
vit_feature_extractor.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.3386 - loss: 1.1043 - val_accuracy: 0.3344 - val_loss: 1.0990
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3502 - loss: 1.1022 - val_accuracy: 0.3344 - val_loss: 1.0987
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.3608 - loss: 1.1137 - val_accuracy: 0.3344 - val_loss: 1.0987
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.3328 - loss: 1.1113 - val_accuracy: 0.3344 - val_loss: 1.0982
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.3192 - loss: 1.1039 - val_accuracy: 0.3344 - val_loss: 1.0981
